In [1]:
import os

current_working_directory = os.getcwd()

print("Current working directory:", current_working_directory)

Current working directory: d:\Python\StockToolsPro\FunctionTest\SwingTradeSignal


安裝pandas庫

In [2]:
#!pip install pandas

  Using cached pandas-2.2.1-cp310-cp310-win_amd64.whl.metadata (19 kB)
  Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl.metadata (61 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.1-cp310-cp310-win_amd64.whl (11.6 MB)
Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl (15.8 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)


將近五天的買入信號和賣出信號提取出來，若有買入信號訊號則把股票代號加入陣列，若有賣出信號訊號則把股票代號加入陣列

In [9]:
import os
import pandas as pd


def check_signals(file_path, day):
    df = pd.read_csv(file_path)
    # 確保日期以降序排序以獲得最後5天
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values('Date', ascending=False, inplace=True)
    
    # 提取最後指定天數
    last_days = df.head(day)
    
    # 檢查買入和賣出信號
    buy_signal = last_days['Buy_Signal'].notna().any()
    sell_signal = last_days['Sell_Signal'].notna().any()
    
    return buy_signal, sell_signal


def recentSignals():
    # 檢查最後5天的買入和賣出信號
    signalsPath = r"D:\Temp\StockData\TW_STOCK_DATA\tradeSignals"
    stockList = os.listdir(signalsPath)
    days = 5
    file_path = os.path.join(signalsPath, stockList[0])
    buy_signal, sell_signal = check_signals(file_path, days)

    buy_stock = []
    sell_stock = []

    if buy_signal:
        buy_stock.append(stockList[0])
    if sell_signal:
        sell_stock.append(stockList[0])

    print("Buy : ", buy_stock)
    print("Sell : ", sell_stock)


if __name__ == "__main__":
    recentSignals()

Buy :  []
Sell :  []


將全部台股近五天的買入信號和賣出信號提取出來，若有買入信號訊號則把股票代號加入陣列，若有賣出信號訊號則把股票代號加入陣列

In [21]:
import os
import pandas as pd


def check_signals(file_path, day):
    df = pd.read_csv(file_path)
    # 確保日期以降序排序以獲得最後指定天數
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values('Date', ascending=False, inplace=True)
    
    # 提取最後指定天數
    last_days = df.head(day)
    
    # 檢查買入和賣出信號
    buy_signal = last_days['Buy_Signal'].notna().any()
    sell_signal = last_days['Sell_Signal'].notna().any()
    
    return buy_signal, sell_signal


def recentSignals():
    # 檢查最後指定天數的買入和賣出信號
    signalsPath = r"D:\Temp\StockData\TW_STOCK_DATA\tradeSignals"
    stockList = os.listdir(signalsPath)
    look_back = 10
    buy_stock = []
    sell_stock = []

    for stock in stockList:
        file_path = os.path.join(signalsPath, stock)
        buy_signal, sell_signal = check_signals(file_path, look_back)

        if buy_signal:
            buy_stock.append(stock.split(".")[0])
        if sell_signal:
            sell_stock.append(stock.split(".")[0])

    print(f"近{look_back}天買入股票清單 : ", buy_stock)
    print(f"近{look_back}天賣出股票清單 : ", sell_stock)


if __name__ == "__main__":
    recentSignals()

近10天買入股票清單 :  ['1103', '1321', '1338', '1446', '1452', '1453', '1464', '1476', '2454', '2465', '2477', '2514', '2901', '3015', '3024', '3356', '6641', '6657', '8454', '9908']
近10天賣出股票清單 :  ['1341', '1418', '1441', '1528', '1529', '1530', '1583', '1597', '1721', '1727', '1808', '2312', '2367', '2387', '2388', '2417', '2428', '2453', '2458', '2462', '2466', '2707', '2762', '3011', '3054', '3058', '3376', '3481', '3504', '4555', '4722', '4927', '4943', '5007', '5222', '5258', '6112', '6115', '6166', '6533', '6789', '6906', '8081', '8996']


把統計好的買入信號和賣出信號輸出合理化

In [27]:
import os
import pandas as pd
from datetime import datetime


def check_signals(file_path, day):
    df = pd.read_csv(file_path)
    # 確保日期以降序排序以獲得最後指定天數
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values('Date', ascending=False, inplace=True)
    
    # 提取最後指定天數
    last_days = df.head(day)
    
    # 檢查買入和賣出信號
    buy_signal = last_days['Buy_Signal'].notna().any()
    sell_signal = last_days['Sell_Signal'].notna().any()
    
    return buy_signal, sell_signal


def extract_signal_details(file_path, day):
    df = pd.read_csv(file_path)
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values('Date', ascending=False, inplace=True)
    last_days = df.head(day)
    buy_signals = last_days[last_days['Buy_Signal'].notna()][['Date', 'Close', 'Buy_Signal']]
    sell_signals = last_days[last_days['Sell_Signal'].notna()][['Date', 'Close', 'Sell_Signal']]
    return buy_signals, sell_signals


def format_output(buy_list, sell_list, buy_details, sell_details, look_back):
    # 格式化股票清單
    buy_list_str = ", ".join(buy_list)
    sell_list_str = ", ".join(sell_list)
    
    buy_output = f"近{look_back}天買入股票清單: [{buy_list_str}]\n"
    sell_output = f"近{look_back}天賣出股票清單: [{sell_list_str}]\n"
    
    # 格式化買入信號詳細資訊
    buy_signals_output = "買入信號詳細資訊:\n"
    for stock, signals in buy_details.items():
        buy_signals_output += f"- 股票代碼 {stock}:\n"
        for _, row in signals.iterrows():
            date = row['Date'].date()  # 直接使用Timestamp對象的date方法
            close_price = f"{row['Close']:.2f}"  # 格式化收盤價至小數點後兩位
            buy_signals_output += f"  日期: {date}, 收盤價: {close_price}, 訊號: {row['Buy_Signal']}\n"
    
    # 格式化賣出信號詳細資訊
    sell_signals_output = "賣出信號詳細資訊:\n"
    for stock, signals in sell_details.items():
        sell_signals_output += f"- 股票代碼 {stock}:\n"
        for _, row in signals.iterrows():
            date = row['Date'].date()  # 直接使用Timestamp對象的date方法
            close_price = f"{row['Close']:.2f}"  # 格式化收盤價至小數點後兩位
            sell_signals_output += f"  日期: {date}, 收盤價: {close_price}, 訊號: {row['Sell_Signal']}\n"
    
    # 結合所有輸出並返回
    return buy_output + sell_output + buy_signals_output + sell_signals_output


def recentSignals():
    # 檢查最後指定天數的買入和賣出信號
    signalsPath = r"D:\Temp\StockData\TW_STOCK_DATA\tradeSignals"
    stockList = os.listdir(signalsPath)
    look_back = 10
    buy_stock = []
    sell_stock = []

    for stock in stockList:
        file_path = os.path.join(signalsPath, stock)
        buy_signal, sell_signal = check_signals(file_path, look_back)

        if buy_signal:
            buy_stock.append(stock.split(".")[0])
        if sell_signal:
            sell_stock.append(stock.split(".")[0])

    # print(f"近{look_back}天買入股票清單 : ", buy_stock)
    # print(f"近{look_back}天賣出股票清單 : ", sell_stock)

    
    # 提取買入和賣出信號的詳細資訊
    buy_signals_details = {}
    sell_signals_details = {}
    for stock in buy_stock:
        file_path = os.path.join(signalsPath, stock + ".TW.csv")
        buy_signals, _ = extract_signal_details(file_path, look_back)
        if not buy_signals.empty:
            buy_signals_details[stock] = buy_signals

    for stock in sell_stock:
        file_path = os.path.join(signalsPath, stock + ".TW.csv")
        _, sell_signals = extract_signal_details(file_path, look_back)
        if not sell_signals.empty:
            sell_signals_details[stock] = sell_signals

    # return buy_signals_details, sell_signals_details
    # print("買入信號詳細資訊 : ", buy_signals_details)
    # print("賣出信號詳細資訊 : ", sell_signals_details)

    # 使用format_output函式格式化輸出
    output = format_output(buy_stock, sell_stock, buy_signals_details, sell_signals_details, look_back)
    print(output)


if __name__ == "__main__":
    recentSignals()

近10天買入股票清單: [1103, 1321, 1338, 1446, 1452, 1453, 1464, 1476, 2454, 2465, 2477, 2514, 2901, 3015, 3024, 3356, 6641, 6657, 8454, 9908]
近10天賣出股票清單: [1341, 1418, 1441, 1528, 1529, 1530, 1583, 1597, 1721, 1727, 1808, 2312, 2367, 2387, 2388, 2417, 2428, 2453, 2458, 2462, 2466, 2707, 2762, 3011, 3054, 3058, 3376, 3481, 3504, 4555, 4722, 4927, 4943, 5007, 5222, 5258, 6112, 6115, 6166, 6533, 6789, 6906, 8081, 8996]
買入信號詳細資訊:
- 股票代碼 1103:
  日期: 2024-03-25, 收盤價: 17.10, 訊號: Buy
- 股票代碼 1321:
  日期: 2024-03-25, 收盤價: 34.50, 訊號: Buy
- 股票代碼 1338:
  日期: 2024-03-25, 收盤價: 35.05, 訊號: Buy
- 股票代碼 1446:
  日期: 2024-03-25, 收盤價: 33.15, 訊號: Buy
- 股票代碼 1452:
  日期: 2024-03-25, 收盤價: 16.25, 訊號: Buy
- 股票代碼 1453:
  日期: 2024-03-25, 收盤價: 14.50, 訊號: Buy
- 股票代碼 1464:
  日期: 2024-03-25, 收盤價: 12.90, 訊號: Buy
- 股票代碼 1476:
  日期: 2024-03-25, 收盤價: 541.00, 訊號: Buy
- 股票代碼 2454:
  日期: 2024-03-25, 收盤價: 1120.00, 訊號: Buy
- 股票代碼 2465:
  日期: 2024-03-25, 收盤價: 112.50, 訊號: Buy
- 股票代碼 2477:
  日期: 2024-03-25, 收盤價: 19.90, 訊號: Buy
- 股票代碼 2514:
  

如何將文件內容轉換為字典結構(tuple結構)，股票代號、股票名稱、產業別

In [48]:
def read_stock_list(file_path):
    stock_info = {}
    with open(file_path, encoding="utf-8") as file:
        for line in file.readlines()[1:]:  # 跳過第一行標題行
            parts = line.strip().split()
            if parts:  # 確保行不是空的
                code, name, market, industry = parts[0], parts[1], parts[2], parts[4]
                stock_info[int(code)] = (name, market, industry)
    return stock_info

stock_info = read_stock_list("D:\\Python\\StockToolsPro\\DataProcess\\CrawlData\\OHLC_DATA\\TW_STOCK_LIST.txt")
print(stock_info)
# print(stock_info[1][0])
# print(stock_info[1][1])
# print(stock_info[1][2])

{1: ('1101', '台泥', '水泥工業'), 2: ('1102', '亞泥', '水泥工業'), 3: ('1103', '嘉泥', '水泥工業'), 4: ('1104', '環泥', '水泥工業'), 5: ('1108', '幸福', '水泥工業'), 6: ('1109', '信大', '水泥工業'), 7: ('1110', '東泥', '水泥工業'), 8: ('1201', '味全', '食品工業'), 9: ('1203', '味王', '食品工業'), 10: ('1210', '大成', '食品工業'), 11: ('1213', '大飲', '食品工業'), 12: ('1215', '卜蜂', '食品工業'), 13: ('1216', '統一', '食品工業'), 14: ('1217', '愛之味', '食品工業'), 15: ('1218', '泰山', '食品工業'), 16: ('1219', '福壽', '食品工業'), 17: ('1220', '台榮', '食品工業'), 18: ('1225', '福懋油', '食品工業'), 19: ('1227', '佳格', '食品工業'), 20: ('1229', '聯華', '食品工業'), 21: ('1231', '聯華食', '食品工業'), 22: ('1232', '大統益', '食品工業'), 23: ('1233', '天仁', '食品工業'), 24: ('1234', '黑松', '食品工業'), 25: ('1235', '興泰', '食品工業'), 26: ('1236', '宏亞', '食品工業'), 27: ('1256', '鮮活果汁-KY', '食品工業'), 28: ('1301', '台塑', '塑膠工業'), 29: ('1303', '南亞', '塑膠工業'), 30: ('1304', '台聚', '塑膠工業'), 31: ('1305', '華夏', '塑膠工業'), 32: ('1307', '三芳', '塑膠工業'), 33: ('1308', '亞聚', '塑膠工業'), 34: ('1309', '台達化', '塑膠工業'), 35: ('1310', '台苯', '塑膠工業'), 36: ('1312', '國喬',

把TW_STOCK_LIST.txt內的股票名稱、產業別加到輸出

In [54]:
import os
import pandas as pd
from datetime import datetime


def check_signals(file_path, day):
    df = pd.read_csv(file_path)
    # 確保日期以降序排序以獲得最後指定天數
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values('Date', ascending=False, inplace=True)
    
    # 提取最後指定天數
    last_days = df.head(day)
    
    # 檢查買入和賣出信號
    buy_signal = last_days['Buy_Signal'].notna().any()
    sell_signal = last_days['Sell_Signal'].notna().any()
    
    return buy_signal, sell_signal


def extract_signal_details(file_path, day):
    df = pd.read_csv(file_path)
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values('Date', ascending=False, inplace=True)
    last_days = df.head(day)
    buy_signals = last_days[last_days['Buy_Signal'].notna()][['Date', 'Close', 'Buy_Signal']]
    sell_signals = last_days[last_days['Sell_Signal'].notna()][['Date', 'Close', 'Sell_Signal']]
    return buy_signals, sell_signals


def read_stock_list(file_path):
    stock_info = {}
    with open(file_path, encoding="utf-8") as file:
        for line in file.readlines()[1:]:  # Skip the first line (header)
            parts = line.strip().split()
            if parts:  # Ensure the line is not empty
                code, name, market = parts[0], parts[1], parts[2]
                industry = parts[4] if len(parts) > 4 else 'Unknown'
                stock_info[int(code)] = (name, market, industry)
    return stock_info


def format_output(buy_list, sell_list, buy_details, sell_details, look_back, stock_info):
    try:
        buy_list_str = ", ".join([f"{stock} {stock_info[int(stock)][1]}({stock_info[int(stock)][2]})" for stock in buy_list])
        sell_list_str = ", ".join([f"{stock} {stock_info[int(stock)][1]}({stock_info[int(stock)][2]})" for stock in sell_list])
    except KeyError as e:
        print(f"KeyError: {e} - This key does not exist in stock_info.")
        return "Error in formatting output."



    buy_output = f"近{look_back}天買入股票清單: [{buy_list_str}]\n"
    sell_output = f"近{look_back}天賣出股票清單: [{sell_list_str}]\n"
    
    # 格式化買入信號詳細資訊
    buy_signals_output = "買入信號詳細資訊:\n"
    for stock, signals in buy_details.items():
        stock_name = stock_info[int(stock)][1]  # 从 stock_info 中获取股票名称
        stock_industry = stock_info[int(stock)][2]  # 获取行业信息
        buy_signals_output += f"- 股票代碼 {stock} {stock_name}({stock_industry}):\n"
        for _, row in signals.iterrows():
            date = row['Date'].date()  # 直接使用Timestamp對象的date方法
            close_price = f"{row['Close']:.2f}"  # 格式化收盤價至小數點後兩位
            buy_signals_output += f"  日期: {date}, 收盤價: {close_price}, 訊號: {row['Buy_Signal']}\n"
    
    # 格式化賣出信號詳細資訊
    sell_signals_output = "賣出信號詳細資訊:\n"
    for stock, signals in sell_details.items():
        stock_name = stock_info[int(stock)][1]  # 从 stock_info 中获取股票名称
        stock_industry = stock_info[int(stock)][2]  # 获取行业信息
        sell_signals_output += f"- 股票代碼 {stock} {stock_name}({stock_industry}):\n"
        for _, row in signals.iterrows():
            date = row['Date'].date()
            close_price = f"{row['Close']:.2f}"
            sell_signals_output += f"  日期: {date}, 收盤價: {close_price}, 訊號: {row['Sell_Signal']}\n"
    
    # 結合所有輸出並返回
    return buy_output + sell_output + buy_signals_output + sell_signals_output


def recentSignals():
    # 檢查最後指定天數的買入和賣出信號
    signalsPath = r"D:\Temp\StockData\TW_STOCK_DATA\tradeSignals"
    stockList = os.listdir(signalsPath)
    look_back = 10
    buy_stock = []
    sell_stock = []

    for stock in stockList:
        file_path = os.path.join(signalsPath, stock)
        buy_signal, sell_signal = check_signals(file_path, look_back)

        if buy_signal:
            buy_stock.append(stock.split(".")[0])
        if sell_signal:
            sell_stock.append(stock.split(".")[0])

    # print(f"近{look_back}天買入股票清單 : ", buy_stock)
    # print(f"近{look_back}天賣出股票清單 : ", sell_stock)

    
    # 提取買入和賣出信號的詳細資訊
    buy_signals_details = {}
    sell_signals_details = {}
    for stock in buy_stock:
        file_path = os.path.join(signalsPath, stock + ".TW.csv")
        buy_signals, _ = extract_signal_details(file_path, look_back)
        if not buy_signals.empty:
            buy_signals_details[stock] = buy_signals

    for stock in sell_stock:
        file_path = os.path.join(signalsPath, stock + ".TW.csv")
        _, sell_signals = extract_signal_details(file_path, look_back)
        if not sell_signals.empty:
            sell_signals_details[stock] = sell_signals

    # return buy_signals_details, sell_signals_details
    # print("買入信號詳細資訊 : ", buy_signals_details)
    # print("賣出信號詳細資訊 : ", sell_signals_details)
    
    # 將文件內容轉換為字典結構(tuple結構)，股票代號、股票名稱、產業別
    stock_info = read_stock_list("D:\\Python\\StockToolsPro\\DataProcess\\CrawlData\\OHLC_DATA\\TW_STOCK_LIST.txt")

    # 使用format_output函式格式化輸出
    output = format_output(buy_stock, sell_stock, buy_signals_details, sell_signals_details, look_back, stock_info)
    print(output)


if __name__ == "__main__":
    recentSignals()

KeyError: 1102 - This key does not exist in stock_info.
Error in formatting output.


計算基於買入和賣出信號的累計交易利潤、正區間利潤數量和負區間利潤數量。
每次交易買入的股數，默認為 1000 股。

In [29]:
import pandas as pd

def calculate_trading_profit(csv_file_path, shares_per_trade=1000):
    """
    計算基於買入和賣出信號的累計交易利潤、正區間利潤數量和負區間利潤數量。

    參數:
    - csv_file_path: CSV 檔案路徑，檔案中應包含交易信號。
    - shares_per_trade: 每次交易買入的股數。默認為 1000 股。

    返回:
    - total_profit: 累計利潤。
    - positive_profit_count: 正區間利潤數量。
    - negative_profit_count: 負區間利潤數量。
    """
    # 讀取數據
    data = pd.read_csv(csv_file_path)

    # 將買入和賣出的非NaN值視為信號
    data['Buy_Signal'] = data['Buy_Signal'].notna()
    data['Sell_Signal'] = data['Sell_Signal'].notna()

    # 初始化變量
    buy_price = 0
    profit_list = []
    in_position = False

    # 迭代每一行，計算交易利潤
    for index, row in data.iterrows():
        if row['Buy_Signal'] and not in_position:
            buy_price = row['Close']  # 設置買入價格
            in_position = True
        elif row['Sell_Signal'] and in_position:
            sell_price = row['Close']  # 設置賣出價格
            profit = (sell_price - buy_price) * shares_per_trade  # 計算利潤
            profit_list.append(profit)  # 添加到利潤列表
            in_position = False  # 重置持倉狀態

    # 計算累計利潤、正利潤數量和負利潤數量
    total_profit = sum(profit_list)
    positive_profit_count = sum(1 for profit in profit_list if profit > 0)
    negative_profit_count = sum(1 for profit in profit_list if profit < 0)

    return total_profit, positive_profit_count, negative_profit_count

# 使用範例
signalsPath = r"D:\Temp\StockData\TW_STOCK_DATA\tradeSignals"
stockList = os.listdir(signalsPath)
csv_file_path = os.path.join(signalsPath, stockList[0])  # 替換為您的 CSV 檔案路徑
total_profit, positive_profit_count, negative_profit_count = calculate_trading_profit(csv_file_path)

print("總利潤: ", total_profit)
print("正利潤計數: ", positive_profit_count)
print("負利潤計數: ", negative_profit_count)


總利潤:  258967.34058856964
正利潤計數:  171
負利潤計數:  5
